# Домашнее задание 2 (5 баллов).

Все задания ниже имеют равный вес (5/16).

In [1]:
import pandas as pd
import os

#### Описание данных

В папке Dat (https://github.com/hse-ds/iad-intro-ds/blob/master/2023/homeworks/Data.zip) находится информация о студентах. Всего 10 групп студентов. Файлы делятся на две категории:
    * Students_info_i - информация о студентах из группы i
    * Students_marks_i - оценки студентов из группы i за экзамены

### Одно из важных достоинств pandas $-$ это удобные методы реляционного взаимодействия с данными, аналогичные, например, возможностям SQL для слияния и конкатенации таблиц: merge, join, concat. Наличие готовых методов позволяет не реализовывать самостоятельно поэлементную обработку данных и оперировать сразу целыми таблицами данных.

Подробно об этих методах посмотрите тут: https://www.kaggle.com/residentmario/renaming-and-combining#Combining

#### Задание 1. Соберите всю информацию о студентах в одну таблицу df. В получившейся таблице должна быть информация и оценки всех студентов из всех групп. Напечатайте несколько строк таблицы для демонстрации результата.¶

In [2]:
direct = 'Data'

#Прямо-таки вычленяем наши 10 csv-файлов с оценками
marks = [os.path.join(direct, file) for file in os.listdir(direct) if 'marks' in file]
marks = [pd.read_csv(file) for file in marks]
marks = pd.concat(marks, ignore_index = True)

#Костыль...
x = marks.iloc[0:100]
y = marks.iloc[100:200]
marks.iloc[0:100] = y
marks.iloc[100:200] = x

#То же самое с инфой
info = [os.path.join(direct, file) for file in os.listdir(direct) if 'info' in file]
info = [pd.read_csv(file) for file in info]
info = pd.concat(info, ignore_index = True)

#И тут костыль (для удобства)...
x = info.iloc[0:100]
y = info.iloc[100:200]
info.iloc[0:100] = y
info.iloc[100:200] = x

#Дабы сократить число строк кода в задании 2
marks.drop(columns = 'index', inplace = True)

#Мёрджим
df = pd.merge(info, marks, left_index=True, right_index=True)
df.head()

,index,gender,race/ethnicity,parental level of education,lunch,test preparation course,group,math score,reading score,writing score
0,0,female,group B,bachelor's degree,standard,none,group1,72,72,74
1,1,female,group C,some college,standard,completed,group1,69,90,88
2,2,female,group B,master's degree,standard,none,group1,90,95,93
3,3,male,group A,associate's degree,free/reduced,none,group1,47,57,44
4,4,male,group C,some college,standard,none,group1,76,78,75


#### Задание 2. Удалите столбец index у полученной таблицы. Напечатайте первые 10 строк таблицы.

In [3]:
df.drop(columns = 'index', inplace = True)
df.head(10)

,gender,race/ethnicity,parental level of education,lunch,test preparation course,group,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,group1,72,72,74
1,female,group C,some college,standard,completed,group1,69,90,88
2,female,group B,master's degree,standard,none,group1,90,95,93
3,male,group A,associate's degree,free/reduced,none,group1,47,57,44
4,male,group C,some college,standard,none,group1,76,78,75
5,female,group B,associate's degree,standard,none,group1,71,83,78
6,female,group B,some college,standard,completed,group1,88,95,92
7,male,group B,some college,free/reduced,none,group1,40,43,39
8,male,group D,high school,free/reduced,completed,group1,64,64,67
9,female,group B,high school,free/reduced,none,group1,38,60,50


#### Задание 3. Выведите на экран размеры полученной таблицы

In [4]:
df.shape

(1000, 9)

#### Задание 4. Выведите на экран статистические характеристики числовых столбцов таблицы (минимум, максимум, среднее значение, стандартное отклонение)

In [5]:
col = ['math score', 'reading score' , 'writing score']
for x in col:
    print(f'{x}:')
    print(df[x].min())
    print(df[x].max())
    print(df[x].mean())
    print(df[x].std())

math score:
0
100
65.358
15.331630536081104
reading score:
17
100
68.544
14.725722763617098
writing score:
10
100
67.423
15.396009486490568


#### Задание 5. Проверьте, есть ли в таблице пропущенные значения

In [6]:
#Сначала создаем булеву серию, а потом там ищем значения True (поэтому два раза "any")
if df.isnull().any(axis = 0).any():
    print('Да')
else:
    print('Нет')

Нет


#### Задание 6. Выведите на экран средние баллы студентов по каждому предмету (math, reading, writing)

In [7]:
col = ['math score', 'reading score' , 'writing score']
for x in col:
    #Сначала выведем средние неокругленные:
    print(f'Среднее {x} до округления: {df[x].mean()}')
    #Потом математическое округление:
    print(f'Среднее {x} целочисленное: {round(df[x].mean())}')

Среднее math score до округления: 65.358
Среднее math score целочисленное: 65
Среднее reading score до округления: 68.544
Среднее reading score целочисленное: 69
Среднее writing score до округления: 67.423
Среднее writing score целочисленное: 67


**Задание 7. Как зависят оценки от того, проходил ли студент курс для подготовки к сдаче экзамена (test preparation course)? Выведите на экран для каждого предмета в отдельности средний балл студентов, проходивших курс для подготовки к экзамену и не проходивших курс.**

In [8]:
col = ['math score', 'reading score' , 'writing score']
for x in col:
    print(f'{x}:')
    compl = df[df['test preparation course'] == 'completed'][x].mean()
    none = df[df['test preparation course'] == 'none'][x].mean()
    print(f'Средний балл проходивших: {compl}')
    print(f'Средний балл не проходивших: {none}')

math score:
Средний балл проходивших: 69.51622418879056
Средний балл не проходивших: 63.22541603630862
reading score:
Средний балл проходивших: 73.58997050147492
Средний балл не проходивших: 65.95612708018155
writing score:
Средний балл проходивших: 74.25958702064896
Средний балл не проходивших: 63.91679273827534


**Задание 8. Выведите на экран все различные значения из столбца lunch.**

In [9]:
print(*df['lunch'].unique())

standard free/reduced


**Задание 9. Переименуйте колонку "parental level of education" в "education", а "test preparation course" в "test preparation" с помощью метода pandas rename**
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html

In [10]:
df = df.rename(columns = {'parental level of education' : 'education'})
df = df.rename(columns = {'test preparation course' : 'test preparation'})

**Зафиксируем минимальный балл для сдачи экзамена**

In [11]:
passmark = 50

**Задание 10. Ответьте на вопросы:**
    * Какая доля студентов сдала экзамен по математике (passmark >= 50)?
    * Какая доля студентов, проходивших курс подготовки к экзамену, сдала экзамен по математике?
    * Какая доля женщин, не проходивших курс подготовки к экзамену, не сдала экзамен по математике? 

In [12]:
pm = passmark

#Ищем долю студентов, у которых значение в math score >= 50
mathpass = len(df[df['math score'] >= pm])
print(f'Ответ на вопрос 1: {mathpass / 1000}')
#Ищем долю студентов, которые закончили курсы и имеют math score >= 50 и делим на количетсво всех посещающих
mathpassprep = len(df[(df['test preparation'] == 'completed') & (df['math score'] >= pm)])
al1 = len(df[df['test preparation'] == 'completed'])
print(f'Ответ на вопрос 2: {mathpassprep / al1}')
#Ищем долю студентов, у которых значения 'female', 'none', и 'math score' >= pm и вычитаем из одного частное от деления этого значения на всех не посещавших женщин
mathpassprepwom = len(df[(df['test preparation'] == 'none') & (df['math score'] >= pm) & (df['gender'] == 'female')])
al2 = len(df[(df['test preparation'] == 'none') & (df['gender'] == 'female')])
print(f'Ответ на вопрос 3: {1 - (mathpassprepwom / al2)}')

Ответ на вопрос 1: 0.852
Ответ на вопрос 2: 0.9233038348082596
Ответ на вопрос 3: 0.21387283236994215


**Задание 11. С помощью groupby выполните задания ниже. Также выведите время выполнения каждого из заданий.**
    * Для каждой этнической группы выведите средний балл за экзамен по чтению
    * Для каждого уровня образования выведите минимальный балл за экзамен по письму

In [13]:
import time

In [14]:
#Задание 1
start = time.time()
print(df.groupby('race/ethnicity')['reading score'].mean())
end = time.time() - start
print(f'Время выполнения: {end}')

race/ethnicity
group A    64.585106
group B    66.613757
group C    68.714734
group D    69.141762
group E    72.386861
Name: reading score, dtype: float64
Время выполнения: 0.007154941558837891


In [15]:
#Задание 2
start = time.time()
print(df.groupby('education')['writing score'].min())
end = time.time() - start
print(f'Время выполнения: {end}')

education
associate's degree    35
bachelor's degree     38
high school           15
master's degree       46
some college          19
some high school      10
Name: writing score, dtype: int64
Время выполнения: 0.003663778305053711


**Задание 12. Выполните задание 11 с помощью циклов. Сравните время выполнения.**

In [16]:
#Задание 1
start = time.time()
nl = df['race/ethnicity'].unique()
for x in nl:
    print(x)
    print(df[df['race/ethnicity'] == x]['reading score'].mean())
end = time.time() - start
print(f'Время выполнения: {end}')

group B
66.6137566137566
group C
68.71473354231975
group A
64.58510638297872
group D
69.14176245210729
group E
72.38686131386861
Время выполнения: 0.009117841720581055


In [17]:
#Задание 2
start = time.time()
nl = df['education'].unique()
for x in nl:
    print(x)
    print(df[df['education'] == x]['writing score'].min())
end = time.time() - start
print(f'Время выполнения: {end}')

bachelor's degree
38
some college
19
master's degree
46
associate's degree
35
high school
15
some high school
10
Время выполнения: 0.008767127990722656


**Задание 13. Выведите на экран средние баллы студентов по каждому предмету в зависимости от пола и уровня образования. То есть должно получиться количество групп, равных 2 * (число уровней образования), и для каждой такой группы выыведите средний балл по каждому из предметов.**

Это можно сделать с помощью сводных таблиц (pivot_table):

https://www.kaggle.com/kamilpolak/tutorial-how-to-use-pivot-table-in-pandas

In [18]:
#Я не знал, что там еще может быть в aggfunc кроме np.mean, поэтому...
import numpy as np

In [19]:
grouped = pd.pivot_table(df, index = ['gender', 'education'],aggfunc = {'writing score': np.mean, 'reading score': np.mean, 'math score': np.mean})
grouped

math score  reading score  writing score
gender education                                                   
female associate's degree   64.520661      73.388430      73.074380
       bachelor's degree    67.210526      76.280702      77.614035
       high school          58.752688      67.827957      66.279570
       master's degree      65.314286      74.571429      75.742857
       some college         66.367521      74.290598      74.931624
       some high school     58.000000      68.305263      67.284211
male   associate's degree   69.062500      66.089286      63.955357
       bachelor's degree    71.396226      68.754717      68.716981
       high school          64.742574      62.198020      59.168317
       master's degree      74.190476      71.952381      71.666667
       some college         67.108108      63.522523      61.630631
       some high school     67.023810      63.416667      60.190476

#### Задание 14. Сколько студентов успешно сдали экзамен по математике?

Создайте новый столбец в таблице df под названием Math_PassStatus и запишите в него F, если студент не сдал экзамен по математике (балл за экзамен < passmark), и P иначе.

Посчитайте количество студентов, сдавших и не сдавших экзамен по математике.

Сделайте аналогичные шаги для экзаменов по чтению и письму.

In [20]:
#Я сделал функцию, которая считывает (через метод apply()) строку и столбец для проверки и заполняет новый столбец значениями по условию
def value(row, column):
    if row[column] < 50:
        return 'F'
    else:
        return 'P'

#После применил эту функцию на все три новых столбца
df['Math_PassStatus'] = df.apply(lambda row: value(row, 'math score'), axis=1)
df['Reading_PassStatus'] = df.apply(lambda row: value(row, 'reading score'), axis=1)
df['Writing_PassStatus'] = df.apply(lambda row: value(row, 'writing score'), axis=1)

#### Задание 15. Сколько студентов успешно сдали все экзамены?

Создайте столбец OverAll_PassStatus и запишите в него для каждого студента 'F', если студент не сдал хотя бы один из трех экзаменов, а иначе 'P'.

Посчитайте количество студентов, которые сдали все экзамены.

In [21]:
#Используем новую функцию. В целом она та же самая, просто с другими условиями и с большим количеством столбцов
def value(row):
    if row['Math_PassStatus'] == 'P' and row['Reading_PassStatus'] == 'P' and row['Writing_PassStatus'] == 'P': 
        return 'P'
    else:
        return 'F'

#Применяем ее на новый столбец. Он один, поэтому функция без дополнительных переменных, и лямбда не нужна
df['OverAll_PassStatus'] = df.apply(value, axis=1)

#### Задание 16. Переведем баллы в оценки

### Система перевода баллов в оценки
####    больше 90 = A
####      80-90 = B
####      70-80 = C
####      60-70 = D
####      50-60 = E
####    меньше 50 = F (Fail)

Создайте вспомогательную функцию, которая будет по среднему баллу за три экзамена выставлять оценку студенту по данным выше критериям.

Создайте столбец Grade и запишите в него оценку каждого студента.

Выведите количество студентов, получивших каждую из оценок.

**В случае, если средний балл попадает на границу между оценками (т.е. равен ровно 60, 70 или 80 баллов), вы можете интерпретировать условие на своё усмотрение (т.е. можете поставить за 60 баллов оценку D, а можете - E).**

In [22]:
#Упростим себе задачу и создадим отдельный столбец со средними баллами по трем экзаменам
df['OverAll_mean'] = df[['math score', 'reading score', 'writing score']].mean(axis=1)

#Делаем функцию на проверку строки
def GetGrade(row):
    if row['OverAll_mean'] < 50:
        return 'F'
    elif row['OverAll_mean'] < 60:
        return 'E'
    elif row['OverAll_mean'] < 70:
        return 'D'
    elif row['OverAll_mean'] < 80:
        return 'C'
    elif row['OverAll_mean'] < 90:
        return 'B'
    else:
        return 'A'
    
#Применяем функцию через метод apply и удаляем столбец со средними (для красоты)
df['Grade'] = df.apply(GetGrade, axis=1)
df.drop(columns = 'OverAll_mean', inplace = True)
df.head()

,gender,race/ethnicity,education,lunch,test preparation,group,math score,reading score,writing score,Math_PassStatus,Reading_PassStatus,Writing_PassStatus,OverAll_PassStatus,Grade
0,female,group B,bachelor's degree,standard,none,group1,72,72,74,P,P,P,P,C
1,female,group C,some college,standard,completed,group1,69,90,88,P,P,P,P,B
2,female,group B,master's degree,standard,none,group1,90,95,93,P,P,P,P,A
3,male,group A,associate's degree,free/reduced,none,group1,47,57,44,F,P,F,F,F
4,male,group C,some college,standard,none,group1,76,78,75,P,P,P,P,C
